# Introduction

In this series of tutorials, the fundamental aspects of a BigDFT calculation are inspected.

The **topics** addressed are:
- building a system
- running a simple calculation
- reading a Logfile

Those subjects are then each followed by **exercises**.

The **prerequisites** are:
- beginner level in python programming (1)
- understanding of *ab initio* methods (2)

In the following, PyBigDFT is used to build systems and then compute their first-principles properties using BigDFT.

The code packages are installed through the following

In [ ]:
install = "client" #@param ["full_suite", "client"]
use_google_drive = False # @param {type:"boolean"}
install_var=install
!wget https://raw.githubusercontent.com/BigDFT-group/bigdft-school/data/packaging/install.py &> /dev/null
args={'locally': True} if not use_google_drive else {}
import install
getattr(install,install_var.split()[0])(**args)

For this part of the tutorial we have to install an extra package.
If your client installation is performed on the google drive, such command will not be needed in the future.


In [ ]:
install.packages('py3Dmol')

## Python intrinsics manipulation

In python, two datastructures are very common: **lists** and **dictionaries**.

In [ ]:
my_list = [0, 1, 2, 3]
print(my_list[-1])
my_dict = {"a": "word", "c": 4}
print(my_dict["c"])

Those objects are easily built and manipulated using comprehensions

In [ ]:
my_list2 = [x*3 for x in my_list]
print(my_list2)
my_dict2 = {k+"2": v for k,v in my_dict.items()}
print(my_dict2)

Additionally, those objects are serializable in yaml format for improved readability

In [ ]:
from yaml import dump

print(dump(my_dict2))

## What do we mean by *ab initio* methods?

Ab initio quantum chemistry methods attempt to solve Schrödinger's equation given the **positions** of the nuclei and the **number of electrons**, yielding useful information such as electron densities, energies and other properties of the system.

A first-principles calculation therefore requires:
- a geometry (along with a lattice for solid-state)
- an exchange-correlation (XC) functional
- a set of input parameters which are specific of the numerical treatment of the method

# The Geometry: Building a system

In PyBigDFT, geometries are build upon different layers:
- Atoms: stores any information (dict)
- Fragments: are collection of Atoms (list)
- Systems: are collection of Fragments (dict)

## Atoms

Any system is composed of atoms, which require both a **symbol** and a **position**.

The most appropriate way to store such information (or any other) about an atom is inside a `dict`

In [ ]:
at = {"sym": "H", "r": [1, 0, 0], "units": "angstroem"}
print(dump(at))

The `Atoms` class wraps up `dict` in order to provide helpful subroutines.

In [ ]:
from BigDFT.Atoms import Atom

atom = Atom(at)
print(dump(atom))

Some of the built in subroutines are demonstrated below.

In [ ]:
print(atom.sym)
print(atom.atomic_number)
print(atom.get_position("angstroem"))
print(atom.get_position("bohr"))

With this approach, the flexibility of a `dict` is retained.

In [ ]:
atom["source"] = "tutorial"
print(atom["source"])
for k,v in atom.items():
    print(k,v)

## Fragments

Calculations involve not single atoms but instead **groups of atoms**. In this case, lists are used as model data structures, with the wrapper class referred to as a `Fragment`.

In [ ]:
at1 = Atom({"sym": "O", "r": [2.3229430273, 1.3229430273, 1.7139430273], "units": "angstroem"})
at2 = Atom({"sym": "H", "r": [2.3229430273, 2.0801430273, 1.1274430273], "units": "angstroem"})
at3 = Atom({"sym": "H", "r": [2.3229430273, 0.5657430273000001, 1.1274430273], "units": "angstroem"})

In [ ]:
from BigDFT.Fragments import Fragment

frag1 = Fragment([at1, at2, at3])
print(len(frag1))
print(frag1.centroid)

It's also possible to build up a fragment in a more step by step process.

In [ ]:
frag1 = Fragment()
frag1.append(at1)
frag1 += Fragment([at2])
frag1.extend(Fragment([at3]))

The fragment properties are then visualized in yaml format

In [ ]:
print(dump(frag1))

## Systems

In PyBigDFT, we have the `System` class at the top, based on a `dict`.
Systems are **named collections of fragments**, with the convention for naming fragments as "NAME:ID" (where name is a string and ID is a number).

In [ ]:
from BigDFT.Systems import System

sys = System()
sys["WAT:0"] = frag1

Similarly, systems are easily readable

In [ ]:
print(dump(sys))

Additional properties can also be displayed, like the **connectivity matrix** and the **unit cell**.

## System visualization and fragments manipulation

It is extremely convenient to visualize `System` objects, just do

In [ ]:
sys.display()

It is equally convenient to manipulate fragments within systems.

Let us **rotate** and **translate** the previous water fragment and add it to the system.

In [ ]:
from copy import deepcopy

frag2 = deepcopy(frag1)
frag2.translate([10, 0, 0])
frag2.rotate(x=90, units="degrees")
sys["WAT:1"] = frag2

In [ ]:
sys.display()

The visualization module has identified that there are two separate fragments, coloring them accordingly (merging fragments would render a uniform visualization). Note for the reader: this was the systems presented in the `test.pdb` file of the QuickStart tutorial.

To summarize the hierarchy, let's iterate over our `System`.

In [ ]:
for fragid, frag in sys.items():
    print(fragid)
    for atm in frag:
        print(dict(atm))

In [ ]:
# atom iteration, if we do not want to keep track of the fragment
for atm in sys.get_atoms():
  print(dict(atm))

## Solid State Systems

The cell attribute of the `System` object enables to investigate systems ranging from **molecular biology** to **condensed matter physics**, by fixing the periodic boundaries conditions.

The `UnitCell` class is available to manage the cell.

In [ ]:
from BigDFT.UnitCells import UnitCell

sys.cell = UnitCell([5, 5, 5], units="bohr")

In [ ]:
print(sys.cell.get_posinp())

BigDFT is able to handle several boundary conditions, depending on the cell.
- if set to `None`: free boundary
- if $x$ and $y$ are set to `inf`: 1D system
- if $y$ is set to `inf`: 2D periodic system (note that $y$ direction is free)
- if all values are `float`: 3D periodic system.

For wire boundary conditions

In [ ]:
sys.cell = UnitCell([float("inf"), float("inf"), 5], units="bohr")
print(sys.cell.get_posinp("bohr"))

For the surface condition

In [ ]:
sys.cell = UnitCell([5, float("inf"), 5], units="bohr")
print(sys.cell.get_posinp("bohr"))

Note that **reduced** (fractional) coordinates can be employed to alternatively specify the locations of atoms (*for fully periodic boundary conditions*).

In [ ]:
cell = UnitCell([10, 10, 10], units="bohr")

In [ ]:
at = Atom({'r': [0.5, 0.25, 0.0], 'sym': "He", 'units': 'reduced'})

print(at.get_position("reduced", cell))
print(at.get_position("bohr", cell))
print(at.get_position("angstroem", cell))

## File I/O

A wide range of standard files can easily be manipulated with PyBigDFT.

### XYZ Files

The `XYZReader` class enables to access the some built in molecules in the database (available [here](https://gitlab.com/l_sim/bigdft-suite/-/tree/devel/PyBigDFT/BigDFT/Database/XYZs)). Otherwise, a path for the filename is required.

In [ ]:
from BigDFT.IO import XYZReader

sys = System()
sys["CH4:0"] = Fragment()
with XYZReader("CH4") as ifile:
    for atom in ifile:
        sys["CH4:0"].append(atom)

sys["CH2F:1"] = Fragment()
with XYZReader("CH2F") as ifile:
    for atom in ifile:
        sys["CH2F:1"].append(atom)

sys["CH2F:1"].translate([-5, 0, 0])

The resulting system is

In [ ]:
sys.display()

Afterwards, the `XYZWriter` class enables to write down our data in the `xyz` format.

In [ ]:
from BigDFT.IO import XYZWriter

natoms = sum([len(x) for x in sys.values()])
with XYZWriter("sys.xyz", natoms=natoms) as ofile:
    for frag in sys.values():
        for at in frag:
            ofile.write(at)

Or, similarly

In [ ]:
from BigDFT import IO

with open('sys.xyz','w') as infile:
    IO.write_xyz(sys,infile)

The advantage of the `XYZreader` (or `XYZwriter`) approach is to directly yield the following attributes: `units`, `natoms` and `cell`.

Warning: when reading an `xyz` file, **there is no fragment information available**

The system is either defined as one fragment (`single`) or each atoms are a single fragment (`atomic`)

**Important**: from the point of view of the BigDFT code, the only important thing is the position of the atoms. The distribution of a system into fragment can be useful for building and (especially) for post-processing purposes. The code will not care about two different fragmentations if the atomic positions are the same.

In [ ]:
with open('sys.xyz','r') as ifile:
    sys_a = IO.read_xyz(ifile,fragmentation="single")
    sys_a.cell=UnitCell()
sys_a.display()

In [ ]:
# look at the defaul name for the fragmentation
list(sys_a)

In [ ]:
with open('sys.xyz','r') as ifile:
    sys_b = IO.read_xyz(ifile,fragmentation="atomic")
sys_b.display()

In [ ]:
# look at the defaul name for the fragmentation
list(sys_b)

### Other Formats

Similarly to `xyz` files, let us write a PDB file (for example).

In [ ]:
with open('sys.pdb', 'w') as ofile:
    IO.write_pdb(sys, ofile)
# the above lines can be replaced by "sys.to_file('sys.pdb')"

Let us then inspect this pdb file

In [ ]:
with open('sys.pdb','r') as ifile:
    for line in ifile:
        print(line, end="")

Of course, a pdb system is also readable

In [ ]:
for fragid, frag in IO.read_pdb(open('sys.pdb','r')).items():
    print(fragid)
    for at in frag:
        print(dict(at))

Notice how the **information on fragments is conserved**

## Exercises on systems

1) Construct a carbon chain of inter-atomic distance of 1.5 angstroem.

2) Construct a complex of C2H4 molecules, arranged in a equilateral triangle, using the molecule database.

3) Construct a graphene lattice using a rectangular cell and a carbon-carbon bond of 1.42 angstroem. (**Advanced**)